# Make Pixel Screens of Pattern Evolution

In [1]:
colors_base = {
"00":"c64_black",  
"01":"c64_white",  
"02":"c64_red",    
"03":"c64_cyan",   
"04":"c64_purple", 
"05":"c64_green",  
"06":"c64_blue",   
"07":"c64_yellow", 
"08":"c64_orange", 
"09":"c64_brown",  
"0a":"c64_ltred",  
"0b":"c64_gray1",  
"0c":"c64_lightgray",  
"0d":"c64_ltgreen",
"0e":"c64_ltblue", 
"0f":"c64_gray3",  
}
colors = [v for k,v in colors_base.items()]
    
c64_to_rgb = {
"c64_black": "#000000",     
"c64_white": "#ffffff",     
"c64_red": "#880000",       
"c64_cyan":  "#aaffee",     
"c64_purple": "#cc44cc",    
"c64_green": "#00cc55",     
#"c64_blue":  "#0000aa",     
"c64_blue":  "#0088ff",     
"c64_yellow":  "#eeee77",   
"c64_orange":  "#dd8855",   
"c64_brown": "#664400",     
"c64_ltred": "#ff7777",  
"c64_gray1":  "#333333",    
"c64_ltgreen": "#aaff66",
"c64_ltblue":  "#0088ff",
"c64_lightgrey": "#bbbbbb", 
"c64_lightgray": "#bbbbbb", 
"c64_gray3": "#bbbbbb",                             
}


Run the following command:
```sh
~/Dev/vice-emu-code/vice/src/x64sc -directory ~/Dev/vice-emu-code/vice/data/ -moncommands painttrace.txt bin/psychedelia.prg 
```
Where `painttrace.txt` is:

```
log on
logname "PsychedeliaIndividualPatterns.txt"
tr exec 0973
command 1 "mem d800 dbff"
```

Read in the ram history.

In [45]:
log_file = "PsychedeliaIndividualPatterns.txt"
input_file = open(log_file,'r')

ram_history = []
lines = input_file.readlines()
c = 0
for i in range(5, len(lines), 67):
    raw_ram = [l[9:60].split() for l in lines[i:i+64]]
    raw_ram = [int(v,16) & 0x0f for l in raw_ram for v in l]
    color_ram = [raw_ram[v:v+40] for v in range(0, len(raw_ram), 40)][:-2]
    ram_history += [color_ram]
    c += 1


Slice up the ram history into the different patterns.

In [76]:
flatten = lambda l: [e for sublist in l for e in sublist]

pattern_breaks = []
for i,h in enumerate(ram_history):
    num_pixels = sum(flatten(h))
    if num_pixels > 1:
        continue
    pattern_breaks += [i]

patterns = []
pp = -1
for p in pattern_breaks:
    if len(ram_history[pp+1:p]) > 1:
        patterns += [ram_history[pp+1:p]]
    pp = p


Methods for culling the pixel data to just the volume that's occupied.

In [60]:
def firstNonZero(l):
    for i,e in enumerate(l):
        if e > 0:
            return i
    return i

def getLeftMostPixelForPattern(pattern):
    return min([firstNonZero(r) for scr in pattern for r in scr])

def getRightMostPixelForPattern(pattern):
    return len(pattern[0][0]) - min([firstNonZero(list(reversed(r)))
                     for scr in pattern for r in scr])

def rotatePatternToCols(pattern):
    pattern_cols = []
    for scr in pattern:
        col_arr = [[0 for i in range(len(scr))] for j in range(len(scr[0]))]
        for i,r in enumerate(scr):
            for j,v in enumerate(r):
                col_arr[j][i] = v
        pattern_cols += [col_arr]
    return pattern_cols

def cullPattern(pattern):
    first_left = getLeftMostPixelForPattern(pattern)
    last_right = getRightMostPixelForPattern(pattern)
    width = last_right - first_left

    pattern_as_cols = rotatePatternToCols(pattern)
    first_top = getLeftMostPixelForPattern(pattern_as_cols)
    last_bottom = getRightMostPixelForPattern(pattern_as_cols)
    height = last_bottom - first_top
    
    culled = [[r[first_left:last_right] 
              for r in scr[first_top:last_bottom]]
              for scr in pattern]
    return culled

#cullPattern(patterns[0])
#patterns[0]

Write a series of png files one for each stage of the evolution of the pattern

In [77]:
from PIL import Image, ImageColor

bit_array = ([['0' for i in range(8)]] 
            + [['0']+['1' for i in range(6)]+['0'] for j in range(6)] 
            + [['0' for i in range(8)]])

def paintChar(origin, pixels, cv):
    c, r = origin
    x_o, y_o = c*8, r*8
    for y, l in enumerate(bit_array):
        for x,bit in enumerate(l):
            if bit == "0":
                continue
            color = c64_to_rgb[colors[cv]]
            pixels[x_o+x, y_o+y] = ImageColor.getrgb(color)


for i,pattern in enumerate(patterns):
    cull_pattern = cullPattern(pattern)
    for j, scr in enumerate(cull_pattern):
        cols=len(scr[0])0
        rows=len(scr)
        bits=8
        img = Image.new( 'RGB', (cols*bits,rows*bits), "black")
        pixels = img.load()
        for y, l in enumerate(scr):
            for x, v in enumerate(l):
                paintChar((x,y), pixels, v)
        img = img.resize((img.width * 4, img.height * 4), Image.NEAREST)
        img.save(f"patterns/pixels/pixel_pattern{i}_{j}.png")



Create top down evolution images

In [84]:
for i,pattern in enumerate(patterns):
    cull_pattern = cullPattern(pattern)
    screen_rows = len(cull_pattern[0])
    screen_cols = len(cull_pattern[0][0])
    cols = len(pattern)
    rows = screen_rows * screen_cols
    bits=8
    img = Image.new( 'RGB', (cols*bits,rows*bits), "black")
    pixels = img.load()
    for x, scr in enumerate(cull_pattern):
        for y2, l in enumerate(scr):
            for x2, v in enumerate(l):
                y = (y2 * screen_cols) + x2
                paintChar((x,y), pixels, v)
    img = img.resize((img.width * 4, img.height * 4), Image.NEAREST)
    img.save(f"patterns/pixels-topdown/pixel_pattern{i}.png")




In [ ]:
segment = ram_history[:300]
screen_rows = len(ram_history[0])
screen_cols = len(ram_history[0][0])
cols = len(segment)
rows = screen_rows * screen_cols
bits=8
img = Image.new( 'RGB', (cols*bits,rows*bits), "black")
pixels = img.load()
for x, scr in enumerate(segment):
    for y2, l in enumerate(scr):
        for x2, v in enumerate(l):
            y = (y2 * screen_cols) + x2
            paintChar((x,y), pixels, v)
#img = img.resize((img.width * 4, img.height * 4), Image.NEAREST)
#img.save(f"patterns/pixels/pixel_pattern{i}_{j}.png")
img
